<a href="https://colab.research.google.com/github/mvenouziou/text_generator/blob/main/poetry_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
# tensorflow modules
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as text  # for loading pre-trained BERT embeddings
import tensorflow_hub as hub

# general modules
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import string

In [10]:
# Split into train/test/validation sets
def load_and_split_csv(source, select_column=False):
    df = pd.read_csv(source)

    if select_column:
        df = df[[select_column]]
    
    X_train, X_test, _, _ = \
        train_test_split(df, df, test_size=0.3, random_state=42)

    X_test, X_valid, _, _ = \
        train_test_split(X_test, X_test, test_size=0.3, random_state=42)

    return X_train, X_test, X_valid

In [3]:
# clean dataset
def clean_text(df, column, dropna=True):
    
    if dropna:
        df.dropna()

    # 'EOL' is used to denote end of line, where the poem line breaks without a separating punctiation)
    df[column] = df[column].apply(lambda x: x.replace('\n', ' EOL '))

    # string helper function
    spaced_punctuation = \
        list(map(lambda x: ' '.join(['', x, '']), string.punctuation))
    # add spacing to separate punctuation from words
    for punc, space_punc in zip(string.punctuation, spaced_punctuation):
        df[column] = df[column].apply(lambda x: x.replace(punc, space_punc))

    # remove extra spaces created in the above processes
    for _ in range(3):
        df[column] = df[column].apply(lambda x: x.replace('  ', ' '))
        
    return None

In [15]:
# word embeddings with BERT pretrained network
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string)
preprocessor = hub.KerasLayer(
    "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/2")
encoder_inputs = preprocessor(text_input)



AttributeError: ignored

In [11]:
data_URL = 'https://raw.githubusercontent.com/mvenouziou' \
            + '/text_generator/main/robert_frost_collection.csv'

X_train, X_test, X_valid = \
    load_and_split_csv(data_URL, select_column = 'Content')

clean_text(X_train, 'Content')   

X_train.head()

,Content
33,"When I go up through the mowing field , EOL Th..."
55,"Thine emulous fond flowers are dead , too , \r..."
69,I had for my winter evening walk - EOL No one ...
28,As I went down the hill along the wall EOL The...
40,"A house that lacks , seemingly , mistress and ..."


In [12]:
X_train

,Content
33,"When I go up through the mowing field , EOL Th..."
55,"Thine emulous fond flowers are dead , too , \r..."
69,I had for my winter evening walk - EOL No one ...
28,As I went down the hill along the wall EOL The...
40,"A house that lacks , seemingly , mistress and ..."
...,...
71,You were forever finding some new play . EOL S...
14,The buzz saw snarled and rattled in the yard E...
92,"Here come the line - gang pioneering by , EOL ..."
51,"Pan came out of the woods one day , — EOL His ..."
